# Ichimoku Cloud Strategy

The Ichimoku Cloud is a comprehensive indicator that defines support and resistance, identifies trend direction, gauges momentum, and provides trading signals. It was developed by Goichi Hosoda, a Japanese journalist, and published in his 1969 book, "Ichimoku Kinko Hyo."

## Key Concepts

- **Conversion Line:** The average of the 9-period high and low. It aims to identify short-term trend changes and smooth out price fluctuations. When the Conversion Line crosses above the Base Line, it signals a bullish trend, and vice versa.

- **Base Line:** The average of the 26-period high and low. It provides confirmation of the trend identified by the Conversion Line. When the Base Line crosses above the Conversion Line, it confirms a bullish trend, and vice versa.

- **Leading Span A:** The average of the Conversion Line and Base Line plotted 26 periods ahead. It acts as a support/resistance level and helps confirm the trend direction.

- **Leading Span B:** The average of the 52-period high and low plotted 26 periods ahead. It provides additional confirmation of the trend direction and acts as a stronger support/resistance level.

- **Lagging Span:** The closing price plotted 26 periods behind. It helps confirm the trend direction and identify potential reversals.

- **Cloud:** The area between Leading Span A and Leading Span B. It represents support/resistance levels and helps identify potential trend changes.

## Trading Signals

- **Bullish Signal:** The price is above the Cloud, the Conversion Line is above the Base Line, and the Cloud is green.

- **Bearish Signal:** The price is below the Cloud, the Conversion Line is below the Base Line, and the Cloud is red.

## Characteristics

- **Trend Identification:** The Ichimoku Cloud helps identify the trend direction and potential reversals.

- **Support/Resistance:** The Cloud, Leading Span A, and Leading Span B act as support/resistance levels.

## Limitations

- **Assumptions:** The Ichimoku Cloud is based on historical price data and may not always predict future price movements accurately.

- **Complexity:** The Ichimoku Cloud consists of multiple components, which can be overwhelming for some traders.

- **False Signals:** Like any technical indicator, the Ichimoku Cloud can generate false signals, leading to losses.

In [ ]:
import sys
!{sys.executable} -m pipenv shell

In [ ]:
!{sys.executable} -m pip install -r requirements.txt --quiet

In [ ]:
!{sys.executable} -m pip list | grep QuantStats

In [ ]:
!{sys.executable} -m pip list | grep pandas

In [5]:
import pandas as pd
import quantstats as qs

In [11]:
%load_ext autoreload
%autoreload 2

## Load Data

We use auxiliar functions from the `data_market` directory to handle these data.

In [ ]:
from data_market.datalake import load_data

dict_data = load_data()

# Let's check the keys of the dictionary, each one a DataFrame
print(dict_data.keys())

In [ ]:
# Let's check the first DataFrame: prices of Brazilian stocks
dict_data['stocks']

## Strategy execution

Following, we test the execution of our strategy: for just a single day.

In [8]:
# You must write all the code for your strategy entirely in the strategy directory
# Your strategy must implement the StrategyInterface interface defined in simulator/strategy_interface.py
from strategy.strategy_ichimoku import IchimokuStrategy

strategy = IchimokuStrategy()

In [9]:
# Execution for a single day
weights = strategy.calculate_next_weights(data = dict_data, t = 500)

# Check if the returned DataFrame has the correct format
assert strategy.check_return(weights)

The strategy must return a DataFrame with the columns: `date`, `ticker` and `weights`.

In [ ]:
weights

## Simulation

Following we simulate our strategy and generate the final report.

In [12]:
from simulator.strategy_simulator import strategy_simulator

# Initialize data structures to store results
ret_port = pd.Series(dtype="float64")  # Series to store portfolio returns
weights_db = pd.DataFrame()  # DataFrame to store historical weights

# Loop through a range of time values
for t in range(500, len(dict_data['stocks'].index) - 1):

    # Use the strategy simulator to get portfolio's historical weights [weights_db]
    # and its next day returns [ret_port]
    ret_port, weights_db = strategy_simulator(path = "results/",
                                              strategy = strategy,
                                              data = dict_data,
                                              t = t,
                                              ret_port = ret_port,
                                              weights_db = weights_db)

## Report

We can use the results of the simulation, saved in the directory `results`, to generate a report of our strategy using `quantstats`.

The simulation generate two parquet files:

- [ret_port.parquet](results/ret_port.parquet): DataFrame with the return of the portfolio, for each simulated datetime
- [wegiths_db.parquet](results/weights_db.parquet): DataFrame with the weights of each stock in the portfolio, for each simulated datetime

In [13]:
ret_port = pd.read_parquet("results/ret_port.parquet")
ret_port['date'] = pd.to_datetime(ret_port['date'], format = "%Y-%m-%d")
ret_port = ret_port.set_index("date").ret_port

In [ ]:
qs.reports.html(ret_port, "^BVSP", text_description="""
    <p>Demonstration of a simple strategy.</p>
    <p><strong>Important</strong>: Transaction costs are not taken into account.</p>""")